In [2]:
import tkinter as tk
from tkinter import messagebox, ttk
import csv
import os
from datetime import datetime
import matplotlib.pyplot as plt

FILE_NAME = "expenses.csv"

# Ensure CSV exists with headers
if not os.path.exists(FILE_NAME):
    with open(FILE_NAME, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Category", "Description", "Amount"])

# ---------------- Functions ----------------
def add_expense():
    date = date_entry.get() or datetime.today().strftime("%Y-%m-%d")
    category = category_entry.get()
    description = desc_entry.get()
    amount = amount_entry.get()

    if not category or not amount:
        messagebox.showerror("Error", "Category and Amount are required!")
        return

    try:
        amount = float(amount)
    except ValueError:
        messagebox.showerror("Error", "Amount must be a number!")
        return

    with open(FILE_NAME, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([date, category, description, amount])

    messagebox.showinfo("Success", "Expense Added!")
    clear_fields()
    view_expenses()

def clear_fields():
    date_entry.delete(0, tk.END)
    category_entry.delete(0, tk.END)
    desc_entry.delete(0, tk.END)
    amount_entry.delete(0, tk.END)

def view_expenses():
    for row in tree.get_children():
        tree.delete(row)
    with open(FILE_NAME, mode="r") as file:
        reader = csv.reader(file)
        next(reader)  # skip header
        for row in reader:
            tree.insert("", tk.END, values=row)

def show_total():
    total = 0
    with open(FILE_NAME, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            total += float(row["Amount"])
    messagebox.showinfo("Total Expenses", f"💰 Total: ₹{total:.2f}")

def monthly_summary():
    month = month_entry.get()
    year = year_entry.get()
    total = 0
    records = []

    with open(FILE_NAME, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row["Date"].startswith(f"{year}-{month}"):
                records.append(row)
                total += float(row["Amount"])

    if records:
        msg = f"📅 Expenses for {month}/{year}:\n"
        for r in records:
            msg += f"{r['Date']} - {r['Category']} - {r['Description']} - ₹{r['Amount']}\n"
        msg += f"\nTotal: ₹{total:.2f}"
        messagebox.showinfo("Monthly Summary", msg)
    else:
        messagebox.showinfo("No Data", "No expenses found for this month!")

def category_report():
    categories = {}
    with open(FILE_NAME, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            categories[row["Category"]] = categories.get(row["Category"], 0) + float(row["Amount"])

    if not categories:
        messagebox.showinfo("No Data", "No expenses to show!")
        return

    # Show chart
    plt.bar(categories.keys(), categories.values(), color="skyblue")
    plt.title("Expenses by Category")
    plt.xlabel("Category")
    plt.ylabel("Amount (₹)")
    plt.show()

def search_expenses():
    keyword = search_entry.get().lower()
    for row in tree.get_children():
        tree.delete(row)
    with open(FILE_NAME, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            if (keyword in row["Category"].lower()) or (keyword in row["Date"]):
                tree.insert("", tk.END, values=[row["Date"], row["Category"], row["Description"], row["Amount"]])

# ---------------- GUI Setup ----------------
root = tk.Tk()
root.title("📊 Personal Expense Tracker")
root.geometry("750x500")
root.config(bg="lightpink")

# Input Section
tk.Label(root, text="Date (YYYY-MM-DD)").grid(row=0, column=0, padx=10, pady=5)
date_entry = tk.Entry(root)
date_entry.grid(row=0, column=1)

tk.Label(root, text="Category").grid(row=1, column=0, padx=10, pady=5)
category_entry = tk.Entry(root)
category_entry.grid(row=1, column=1)

tk.Label(root, text="Description").grid(row=2, column=0, padx=10, pady=5)
desc_entry = tk.Entry(root)
desc_entry.grid(row=2, column=1)

tk.Label(root, text="Amount").grid(row=3, column=0, padx=10, pady=5)
amount_entry = tk.Entry(root)
amount_entry.grid(row=3, column=1)

tk.Button(root, text="Add Expense", command=add_expense, bg="lightgray").grid(row=4, column=0, pady=10)
tk.Button(root, text="View Expenses", command=view_expenses, bg="lightgray").grid(row=4, column=1, pady=10)

# Table
columns = ("Date", "Category", "Description", "Amount")
tree = ttk.Treeview(root, columns=columns, show="headings", height=10)
for col in columns:
    tree.heading(col, text=col)
tree.grid(row=5, column=0, columnspan=5, padx=10, pady=10)

# Extra Features
tk.Button(root, text="Show Total", command=show_total, bg="lightgray").grid(row=6, column=0, pady=10)
tk.Label(root, text="Month (MM)").grid(row=7, column=0)
month_entry = tk.Entry(root, width=5)
month_entry.grid(row=7, column=1)

tk.Label(root, text="Year (YYYY)").grid(row=7, column=2)
year_entry = tk.Entry(root, width=8)
year_entry.grid(row=7, column=3)
tk.Button(root, text="Monthly Summary", command=monthly_summary, bg="yellow").grid(row=7, column=4, padx=10)

tk.Button(root, text="Category Report", command=category_report, bg="pink").grid(row=8, column=0, pady=10)

# Search
import datetime

def search_by_month(month, year):
    results = []
    with open("expenses.csv", "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            try:
                # Convert string date to datetime object
                date_obj = datetime.datetime.strptime(row["Date"], "%d-%m-%Y")
                
                # Match month & year
                if date_obj.month == int(month) and date_obj.year == int(year):
                    results.append(row)
            except:
                pass
    return results
def reset_data():
    with open(FILE_NAME, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Category", "Description", "Amount"])
    messagebox.showinfo("Reset", "All expenses deleted! Starting fresh.")
    view_expenses()

# Add reset button in GUI
tk.Button(root, text="Reset Data", command=reset_data, bg="lightgray").grid(row=10, column=0, pady=10)

def save_data():
    with open(FILE_NAME.get(),mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Category", "Description", "Amount"]) #header row
    messagebox.showinfo("save", "All expenses deleted! Starting fresh.")
    view_expenses()
    
# Add save button in GUI
tk.Button(root, text="save data",command=save_data, bg="lightgray").grid(row=10,column=1,pady=10)
                                        
root.mainloop()